# GitHub Search with Existing Browser Connection

This example demonstrates using webtask's autonomous agent to search GitHub and extract results, all while connected to your existing Chrome browser.

## Setup

Before running this notebook, launch Chrome with debugging enabled:

```bash
# On Mac/Windows:
chrome --remote-debugging-port=9222 --user-data-dir=~/Desktop/chrome-automation

# On Linux:
google-chrome --remote-debugging-port=9222 --user-data-dir=~/Desktop/chrome-automation
```

## Requirements

- Chrome/Chromium launched with `--remote-debugging-port=9222`
- `GOOGLE_API_KEY` environment variable set

In [1]:
# Imports
import asyncio
import os
from webtask import Webtask
from webtask.integrations.llm import GeminiLLM
from webtask.integrations.browser.playwright import PlaywrightBrowser

/home/stevewang/Github/web-auto/webtask/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Connect to Existing Browser

Connect to your Chrome browser that's already running with debugging enabled.

In [2]:
# Connect to existing Chrome browser
print("📡 Connecting to existing Chrome browser...")
browser = await PlaywrightBrowser.connect("http://localhost:9222")
print("✅ Connected!")

📡 Connecting to existing Chrome browser...
✅ Connected!


## Create Agent

Create a webtask agent that will control your browser autonomously.

In [3]:
# Create LLM
api_key = os.getenv("GOOGLE_API_KEY")
llm = GeminiLLM.create(model="gemini-2.5-flash", api_key=api_key)

# Create Webtask and agent using the connected browser
wt = Webtask()
agent = await wt.create_agent_with_browser(
    llm=llm,
    browser=browser,
 )

print("🤖 Agent ready!")

🤖 Agent ready!


## Execute Task

Now let the agent autonomously search GitHub for 'playwright' and extract the top result.

Watch your Chrome window - you'll see the agent navigate, search, and read results automatically!

In [4]:
# Execute autonomous task
print("🔍 Executing task: Search GitHub for 'playwright' and get the top result...\n")

result = await agent.execute(
    task_description="Go to GitHub.com, search for 'playwright', and tell me the name and description of the top repository result"
)

print("\n" + "="*60)
print("📊 Task Result:")
print("="*60)
print(f"Status: {result.status}")
print(f"\nSteps taken: {len(result.steps)}")
print(f"\nFinal answer:\n{result.final_answer}")
print("="*60)

🔍 Executing task: Search GitHub for 'playwright' and get the top result...

2025-11-18 15:09:49 - webtask._internal.agent.worker.worker - INFO - Worker session start - Task: Go to GitHub.com, search for 'playwright', and tell me the name and description of the top repository result
2025-11-18 15:09:49 - webtask._internal.agent.worker.worker - INFO - Step 1 - Start
2025-11-18 15:09:51 - webtask.llm.llm - INFO - Token usage - Prompt: 1174, Response: 30, Total: 1222
2025-11-18 15:09:51 - webtask._internal.agent.worker.worker - INFO - Received LLM response - Tools: ['navigate', 'wait']


Error: BrowserContext.new_cdp_session: page: no object with guid page@72f9a16b809123f0fd9c6316eb904491

## View Execution History

See what the agent did step-by-step.

In [ ]:
# Show step-by-step execution
print("\n📝 Execution Steps:\n")
for i, step in enumerate(result.steps, 1):
    print(f"Step {i}:")
    if hasattr(step, 'proposal') and step.proposal:
        print(f"  Thinking: {step.proposal.thinking}")
        if step.proposal.actions:
            for action in step.proposal.actions:
                print(f"  Action: {action}")
    print()

## Cleanup

Disconnect from the browser (but keep it running).

In [ ]:
# Cleanup: Close the Playwright connection (but not the customer's browser!)
await browser.close()
print("👋 Disconnected. Your browser stays open.")